In [35]:
import os
from pathlib import Path
import sys

# Set the "./../" from the script folder
dir_name = None
try:
    dir_name = os.path.dirname(os.path.realpath(__file__))
except NameError:
    print("WARNING: __file__ not found, trying local")
    dir_name = os.path.abspath('')
lib_path = os.path.realpath(f"{Path(dir_name).parent}")
# Add to path
if lib_path not in sys.path:
    print(f"Adding library path: {lib_path} to PYTHONPATH")
    sys.path.append(lib_path)
else:
    print(f"Library path {lib_path} already in PYTHONPATH")

import utilities

Library path /home/vineeth/Desktop/III-II/SE/project/automated-refactoring already in PYTHONPATH


In [36]:
import json

In [66]:
def build_prompt(prompt_file_path, checkstyle_output_path, current_code_path):
    with open(prompt_file_path, 'r') as json_file:
        prompt_file = json.load(json_file)

    considered_code_path = current_code_path.split("temp")[1][1:]

    prompt = prompt_file["base"] + prompt_file["code_desc"].format(file_name = considered_code_path)

    with open(current_code_path) as f:
        current_code = f.read()

    # adding code
    prompt += "```java\n"
    prompt += current_code
    prompt += "\n```\n\n"

    # add code metrics
    prompt += prompt_file["checkstyle_desc"]

    with open(checkstyle_output_path) as f:
        all_code_metrics = f.readlines()
    # only considering metrics for current file
    code_file_name = os.path.basename(current_code_path)
    current_code_metrics = ""
    for i in all_code_metrics:
        if code_file_name in i:
            current_code_metrics += " - "
            current_code_metrics += i
    current_code_metrics += "\n"

    if current_code_metrics == "\n":
        current_code_metrics = "No smells were found\n"

    prompt += current_code_metrics

    # adding description of smells
    prompt += prompt_file["checkstyle_metrics"]

    # adding ending
    prompt += prompt_file["prompt_ending"]

    return prompt

In [68]:
prompt = build_prompt("../base_prompt.json", "../temp/code-smells.txt", "../temp/refactoring-test-repo-main/books-web/src/main/java/com/sismics/books/rest/resource/UserResource.java")

with open("example_prompt.txt", "w") as f:
    f.write(prompt)